Let's mock a logical error rate evaluation where the logical error rate is computed by $p_L = A \frac{p}{p_{\text{th}}}^{\frac{d+1}{2}}$ and the speed of the simulation scales with the number of syndromes (similar to that of an MWPM decoder).

In [ ]:
import scipy.stats as stats
import distributed
import time

def mock_logical_error_rate(shots: int, d: int, p: float, pth: float = 0.1, A: float = 0.1, per_defect_time: float = 1e-4) -> LogicalErrorResult:
    # distributed.print(f"running {shots} of d={d}, p={p}")
    pL = min(1, A * (p / pth) ** ((d + 1) // 2))
    binom_dist = stats.binom(shots, pL)
    logical_errors = binom_dist.rvs()  # return a random sample of how many logical errors
    physical_errors = d * d * p * shots
    time.sleep(physical_errors * per_defect_time)
    return LogicalErrorResult(logical_errors)

In [ ]:
from IPython import display
import numpy as np
import matplotlib.pyplot as plt
import sinter
import math
from typing import Iterable, Callable
from dataclasses import dataclass, field

d_vec = [3, 5, 7, 9]
p_vec = [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3]
jobs = [
    MonteCarloJob(d=d, p=p) for d in d_vec for p in p_vec
]

executor = MonteCarloJobExecutor(
    client,
    mock_logical_error_rate,
    jobs,
    filename="monte-carlo-sampling-on-cluster.json",
)

def min_shots_submitter(jobs: Iterable[MonteCarloJob], min_shots: int = 10000) -> list[tuple[MonteCarloJob, int]]:
    return [(job, min_shots - job.expecting_shots) for job in jobs if job.expecting_shots < min_shots]

executor.execute(min_shots_submitter, loop_callback=IntermediatePlotter())

We can then try something more intelligent: instead of using a constant number of shots, how about let's try to make the error bar similar? That is equivalent to accumulating sufficient number of errors, e.g., 100 errors

In [ ]:
from dataclasses import dataclass

def intelligent_submitter(jobs: Iterable[MonteCarloJob]) -> list[tuple[MonteCarloJob, int]]:
    submit = min_shots_next_jobs(jobs, min_shots = 10000)
    submit += LimitedTimePrecisionSubmitter()(jobs)
    return submit


d_vec = [3, 5, 7, 9]
p_vec = [0.1 ** (i / 8) for i in range(0, 24)]
jobs = [
    MonteCarloJob(d=d, p=p) for d in d_vec for p in p_vec
]

executor = MonteCarloJobExecutor(
    Client(cluster),
    mock_logical_error_rate,
    jobs,
    filename="monte-carlo-sampling-on-cluster.json",
)

plotter = IntermediatePlotter()
progress_plotter = JobProgressPlotter()
def callback(executor: MonteCarloJobExecutor):
    plotter(executor)
    progress_plotter(executor)
executor.execute(intelligent_submitter, loop_callback=callback)

I want to know more about the running jobs, what about a table?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig_border = 'skyblue'
data =  [
            [         'India', 'China', 'Russia', 'USA', 'Australia'],
            [ '1980',  696828385,982372466,  138257420,  223140018,  14706322],
            ['1990',  870452165, 1153704252,  148005704,   248083732, 17048003],
            ['2000',  1059633675,  1264099069,  146844839,  282398554, 19017963],
            ['2010',  1240613620,  1348191368,  143242599,  311182845, 22019168],
            ['2020', 1396387127, 1424929781, 145617329,  335942003,  25670051],
            [ '1980',  696828385,982372466,  138257420,  223140018,  14706322],
            ['1990',  870452165, 1153704252,  148005704,   248083732, 17048003],
            ['2000',  1059633675,  1264099069,  146844839,  282398554, 19017963],
            ['2010',  1240613620,  1348191368,  143242599,  311182845, 22019168],
            ['2020', 1396387127, 1424929781, 145617329,  335942003,  25670051],
            [ '1980',  696828385,982372466,  138257420,  223140018,  14706322],
            ['1990',  870452165, 1153704252,  148005704,   248083732, 17048003],
            ['2000',  1059633675,  1264099069,  146844839,  282398554, 19017963],
            ['2010',  1240613620,  1348191368,  143242599,  311182845, 22019168],
            ['2020', 1396387127, 1424929781, 145617329,  335942003,  25670051],
            [ '1980',  696828385,982372466,  138257420,  223140018,  14706322],
            ['1990',  870452165, 1153704252,  148005704,   248083732, 17048003],
            ['2000',  1059633675,  1264099069,  146844839,  282398554, 19017963],
            ['2010',  1240613620,  1348191368,  143242599,  311182845, 22019168],
        ]
#the headers from the data array
column_headers = data.pop(0)
row_headers = [x.pop(0) for x in data]

cell_text = []
for row in data:
    cell_text.append([f'{x/1000:1.1f}' for x in row])


print( plt.cm.BuPu(0.1))
# rcolors = plt.cm.BuPu(np.full(len(row_headers), 0.1))
# ccolors = plt.cm.BuPu(np.full(len(column_headers), 0.1))

#Creating the figure. Setting a small pad on the tight layout

plt.figure(tight_layout={'pad':0})

#Adding a table at the bottom of the axes

the_table = plt.table(cellText=cell_text,
                      rowLabels=row_headers,
                      rowColours=rcolors,
                      rowLoc='right',
                      colColours=ccolors,
                      colLabels=column_headers,
                      loc='center')
the_table.auto_set_column_width(list(range(5)))

# Hiding axes
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# Hiding axes border
plt.box(on=None)

In [ ]:
from concurrent.futures import Future

future = Future()
future.set_result(0)
future.done()


In [ ]:
future = Future()
future.done()

In [ ]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(
    queue='regular',
    account="myaccount",
    cores=24,
    memory="500 GB"
)
cluster.scale(jobs=10)  # ask for 10 jobs